In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy import stats
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install xlsxwriter
pd.options.mode.chained_assignment = None  # default='warn'
import statsmodels.api as sm
from statsmodels.formula.api import ols
## https://www.kaggle.com/code/alexmaszanski/two-way-anova-with-python/notebook
from scipy.stats import linregress
import glob
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import re
from statsmodels.stats.anova import AnovaRM
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pingouin as pg



DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


**There are four assumptions that must be met before using two-way ANOVA:**

- Normality: Observations from the sample population are normally distributed.
- Sample Size: The number of observations must be the same for each group.
- Equal Variances: The variances for each group are equal.
- Independence: Observations in each group are independent.

In [4]:
def custom_sort(item):
    if item.endswith('basal'):
        return 0
    else:
        return int(item.split('uM')[0].split('_')[-1])

In [13]:
currdir= os.getcwd()
parent = os.path.dirname(currdir)
gparent=os.path.dirname(parent)

lines_to_skip = 10 # adjust 
plate_type = 'Div19' #adjust
EXPERIMENT_TIME = "1_MIN"
PERTURBATION = "CNO" # adjust
firstTableHeading = "Number of Spikes"
## conditions taken: ### we will define this as dose1,dose2, dose3, dose4, each incrementing in terms of doses

FOLDER_PATH = f"{parent}/example_data/Div19_CNO/Div19_1minute" #
folder_dict = {}
balances= {"basal": "",
        "dose1": "",
        "dose2": "",
        "dose3": "",
        "dose4": ""
        }

## WALK Through folder of interest
for dirpath, dirnames, filenames in os.walk(FOLDER_PATH):
    if dirpath != FOLDER_PATH:
        folder_dict[dirpath.split("/")[-1]] = dirpath
    elif not dirnames:
        folder_dict['basal'] = dirpath

## get doses names in increasing order 
dose_names = [name for name in folder_dict.keys() if 'basal' not in name]
dose_names.sort(key=custom_sort)

# Initialize balances dictionaryd
balances = {"basal": "basal"}

## map sorted doses
for i, dose_name in enumerate(dose_names, start=1):
    balances[f"dose{i}"] = dose_name.split("_")[-1]

## repeated measures ANOVA test?
ANOVARM = True

print("Balances:", balances)
folder_dict

Balances: {'basal': 'basal', 'dose1': '5uMCNO', 'dose2': '10uMCNO', 'dose3': '15uMCNO', 'dose4': '20uMCNO'}


{'Div19_5uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/example_data/Div19_CNO/Div19_1minute/Div19_5uMCNO',
 'Div19_10uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/example_data/Div19_CNO/Div19_1minute/Div19_10uMCNO',
 'Div19_basal': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/example_data/Div19_CNO/Div19_1minute/Div19_basal',
 'Div19_20uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/example_data/Div19_CNO/Div19_1minute/Div19_20uMCNO',
 'Div19_15uMCNO': '/home/poojaparameswaran/Documents/SoderlingLab/MultipleElectrodeAnalysisANOVA/example_data/Div19_CNO/Div19_1minute/Div19_15uMCNO'}

In [14]:
def read_csvs_to_dict(folder, lines_to_skip=10):
    files = glob.glob(os.path.join(folder, "*.csv"))
    data_dict = {}
    for file in files:
        # Extracting the plate number from the filename
        cond = (file.split("/")[-2].split("_")[-1])
        plate_num = re.search(r'plate\s?(\d+)', file, re.IGNORECASE)
        if plate_num:
            plate_key = f"plate_{plate_num.group(1)}"
            df = pd.read_csv(file, skiprows = lines_to_skip)
            df = df.drop(columns=['Unnamed: 9'])
            df['Subject'] = plate_key
            data_dict[plate_key] = df
    return data_dict
all_files_dict = {}

for dose, folder in folder_dict.items():
    all_files_dict[dose] = read_csvs_to_dict(folder)
    
for dose, files_dict in all_files_dict.items():
    print(files_dict.keys())

dict_keys(['plate_1', 'plate_3'])
dict_keys(['plate_1', 'plate_3'])
dict_keys(['plate_1', 'plate_3'])
dict_keys(['plate_3', 'plate_1'])
dict_keys(['plate_3', 'plate_1'])


In [15]:
def concat_plates(plates_dict, firstHeading):
    experiment = {}
    testparams_concat = {}
    for plate, df in plates_dict.items():
        maxrows = df.shape[0]
        vals_between_tables = 16
        titles = [heading_ind for heading_ind in range(14, maxrows+16, 16)]
        titles.insert(0,0)
        # # Empty dictionary to store dataframes
        # Loop through start indices
        for ind in range(len(titles) -1):
            if ind == 0:
                ## Handle mean firing rate
                key = firstHeading
                table = df.iloc[titles[ind] : titles[ind +1], :]
                # table = rename_row_names(table)
            elif ind != 0:
                table = df.iloc[titles[ind] : titles[ind +1], :]
                key = table.iloc[0][0] # just get name
                table = df.iloc[titles[ind] +2: titles[ind +1], :]

            table.set_index('Unnamed: 0', inplace=True)
            if key not in experiment:
                experiment[key] = []
            experiment[key].append(table)
    
    for testparam, df_list in experiment.items():
        concated_table = pd.concat(df_list, ignore_index=False)
        testparams_concat[testparam] = concated_table
    
    testparams_concat = {key.strip(): value for key, value in testparams_concat.items()}
    return testparams_concat
doses = {}
for dose, plates in all_files_dict.items():
    doses[dose] = concat_plates(plates, firstTableHeading)

doses.keys()

dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO'])

In [16]:
def remove_mean_SEM(test_dict, chemical_type):
    for key, value in test_dict.items():
        to_drop = [index for index in ['B Mean', 'B SEM'] if index in value.index]
        if to_drop:
            test_dict[key] = value.drop(index=to_drop)
        test_dict[key]['Chemical_Type']=chemical_type
    return test_dict
for dose, parameter_dict in doses.items():
    doses[dose] = remove_mean_SEM(parameter_dict, dose)

doses.keys()

dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO'])

In [17]:
def rename_row_names(experiment_df):
    # 'Unnamed: 0' is a column
    experiment_df = experiment_df.reset_index()
    experiment_df = experiment_df.drop(columns="Unnamed: 0", errors="ignore")
    plates = list(experiment_df['Subject'])
    chems = list(experiment_df['Chemical_Type'].apply(lambda x: x.split("_")[-1]))
    experiment_df.index = [f"Replicate{i+1}" for i in range(len(experiment_df))]
    experiment_df.drop(columns='Subject', inplace=True)
    return experiment_df
for dose, test_dict in doses.items():
    for test_name, df in test_dict.items():
        test_dict[test_name] = rename_row_names(df)
    doses[dose] = test_dict


In [18]:
doses['Div19_5uMCNO']['Number of Spikes']

,GPR37L 1,GPR37L 2,LGI2 A12,SLITRK5,THSD7 1,THSD7 2,unt1,unt2,Chemical_Type
Replicate1,3814.000000,915.000000,4853.000000,1634.000000,2229.000000,4995.000000,1025.000000,945.000000,Div19_5uMCNO
Replicate2,3359.000000,562.000000,5386.000000,3747.000000,3326.000000,3341.000000,1936.000000,2184.000000,Div19_5uMCNO
Replicate3,1691.000000,2968.000000,2112.000000,2576.000000,2736.000000,0.000000,1610.000000,2023.000000,Div19_5uMCNO
Replicate4,196.000000,589.000000,1244.000000,2880.000000,3488.000000,229.000000,2600.000000,2951.000000,Div19_5uMCNO
Replicate5,0.000000,2322.000000,1848.000000,3369.000000,0.000000,1543.000000,2244.000000,694.000000,Div19_5uMCNO
Replicate6,176.000000,1631.000000,739.000000,2943.000000,471.000000,668.000000,1869.000000,33.000000,Div19_5uMCNO
Replicate7,967.000000,1112.000000,713.000000,1118.000000,2215.000000,488.000000,2630.000000,4112.000000,Div19_5uMCNO
Replicate8,0.000000,4532.000000,2565.000000,62.000000,1559.000000,765.000000,1014.000000,806.000000,Div19_5uMCNO
Replicate9,246.000000,2876.000000,664.000000,614.000000,2410.000000,2051.000000,2020.000000,956.000000,Div19_5uMCNO
Replicate10,1760.000000,4045.000000,736.000000,2294.000000,38.000000,2198.000000,1346.000000,661.000000,Div19_5uMCNO


In [19]:
balances.keys(), balances.values(), doses.keys()

(dict_keys(['basal', 'dose1', 'dose2', 'dose3', 'dose4']),
 dict_values(['basal', '5uMCNO', '10uMCNO', '15uMCNO', '20uMCNO']),
 dict_keys(['Div19_5uMCNO', 'Div19_10uMCNO', 'Div19_basal', 'Div19_20uMCNO', 'Div19_15uMCNO']))

In [21]:
dmapped = {}
for bkey, bval in balances.items():
    for dkey, dval in doses.items():
        if bval == dkey.split("_")[-1]:
            dmapped[bkey] = dval


list(map(lambda x: dmapped[x]['Mean Firing Rate (Hz)']['Chemical_Type'].unique(), dmapped.keys()))

[array(['Div19_basal'], dtype=object),
 array(['Div19_5uMCNO'], dtype=object),
 array(['Div19_10uMCNO'], dtype=object),
 array(['Div19_15uMCNO'], dtype=object),
 array(['Div19_20uMCNO'], dtype=object)]

In [22]:
def combine_chem_types(dict_of_alldicts):
    all_concat = {key: None for key in dict_of_alldicts['basal']}
    for type in dict_of_alldicts.keys():
        for test_parameter, df in dict_of_alldicts[type].items():
            all_concat[test_parameter]=pd.concat([all_concat[test_parameter], df], ignore_index=False)
    for testparam, df in all_concat.items():
        df.columns = df.columns.str.replace(' ', '_')
        for gene in df.columns:
            if gene != "Chemical_Type" and gene !="Subject":
                df[gene]= df[gene].astype(float)
    return all_concat

tests_mapped = combine_chem_types(dmapped)
print(tests_mapped['Network Burst Duration - Avg (sec)']['Chemical_Type'].unique())
tests_mapped['Burst Duration - Avg (sec)']

['Div19_basal' 'Div19_5uMCNO' 'Div19_10uMCNO' 'Div19_15uMCNO'
 'Div19_20uMCNO']


,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,0.186727,0.186965,0.137672,0.092251,0.120965,0.127955,0.230830,0.130939,Div19_basal
Replicate2,0.104390,0.135870,NaN,0.224632,0.116188,NaN,0.178134,0.138585,Div19_basal
Replicate3,0.109907,0.148396,0.270000,0.137095,NaN,NaN,0.218353,NaN,Div19_basal
Replicate4,0.128632,0.169141,0.144687,0.114523,0.104324,0.083313,0.139580,0.216390,Div19_basal
Replicate5,0.095125,0.162516,0.081645,0.253086,NaN,NaN,0.128919,0.132584,Div19_basal
...,...,...,...,...,...,...,...,...,...
Replicate20,0.184312,0.154091,0.365938,0.391940,0.114314,0.141177,0.169773,0.493080,Div19_20uMCNO
Replicate21,0.260519,0.218340,0.323215,0.137296,0.190083,0.183151,0.121089,0.248830,Div19_20uMCNO
Replicate22,0.191750,0.239889,0.253960,0.128927,0.273050,0.185235,0.137482,0.203827,Div19_20uMCNO
Replicate23,0.212670,0.153942,0.224128,0.149553,0.217632,0.152794,0.162906,0.234970,Div19_20uMCNO


In [23]:
def impute_nans(testparam, table):
    chem_types = table['Chemical_Type']
    table = table.drop(columns=['Chemical_Type'])
    original_index = table.index
    imputer = IterativeImputer(max_iter=100, random_state=0)
    imputed_data = imputer.fit_transform(table)
    # Convert the imputed data back to a DataFrame
    imputed_df = pd.DataFrame(imputed_data, columns=table.columns, index=original_index)
    imputed_df['Chemical_Type'] = chem_types
    return imputed_df

# if nans in all columns, skip. need to remove chemtype and subject bc these will never have nans
testsI_mapped = {test_param: impute_nans(test_param, table) for test_param, table in tests_mapped.items() \
    if not table.loc[:, ~(table.columns.isin(['Chemical_Type']))].isna().all().all()
} 
testsI_mapped['Burst Duration - Avg (sec)']

,GPR37L_1,GPR37L_2,LGI2_A12,SLITRK5,THSD7_1,THSD7_2,unt1,unt2,Chemical_Type
Replicate1,0.186727,0.186965,0.137672,0.092251,0.120965,0.127955,0.230830,0.130939,Div19_basal
Replicate2,0.104390,0.135870,0.162157,0.224632,0.116188,0.183760,0.178134,0.138585,Div19_basal
Replicate3,0.109907,0.148396,0.270000,0.137095,0.186200,0.197760,0.218353,0.221479,Div19_basal
Replicate4,0.128632,0.169141,0.144687,0.114523,0.104324,0.083313,0.139580,0.216390,Div19_basal
Replicate5,0.095125,0.162516,0.081645,0.253086,0.155419,0.186325,0.128919,0.132584,Div19_basal
...,...,...,...,...,...,...,...,...,...
Replicate20,0.184312,0.154091,0.365938,0.391940,0.114314,0.141177,0.169773,0.493080,Div19_20uMCNO
Replicate21,0.260519,0.218340,0.323215,0.137296,0.190083,0.183151,0.121089,0.248830,Div19_20uMCNO
Replicate22,0.191750,0.239889,0.253960,0.128927,0.273050,0.185235,0.137482,0.203827,Div19_20uMCNO
Replicate23,0.212670,0.153942,0.224128,0.149553,0.217632,0.152794,0.162906,0.234970,Div19_20uMCNO


In [24]:
def melt_df_Long(df):
    df = df.reset_index().rename(columns={'index': 'Subject'})
    new_df = pd.melt(df, id_vars=['Subject','Chemical_Type'], var_name='Gene', value_name='Quantity')

    return new_df

# Apply the function to your data
testsI_long = {test_param: melt_df_Long(table) for test_param, table in testsI_mapped.items()}


In [25]:
testsI_long['Number of Active Electrodes']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0
1,Replicate2,Div19_basal,GPR37L_1,8.0
2,Replicate3,Div19_basal,GPR37L_1,8.0
3,Replicate4,Div19_basal,GPR37L_1,8.0
4,Replicate5,Div19_basal,GPR37L_1,8.0
...,...,...,...,...
955,Replicate20,Div19_20uMCNO,unt2,8.0
956,Replicate21,Div19_20uMCNO,unt2,8.0
957,Replicate22,Div19_20uMCNO,unt2,8.0
958,Replicate23,Div19_20uMCNO,unt2,8.0


In [26]:
## REMOVE UNT1
for testparam, table in testsI_long.items():
    testsI_long[testparam] = table[table['Gene'] != 'unt1']
testsI_long['Number of Active Electrodes']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0
1,Replicate2,Div19_basal,GPR37L_1,8.0
2,Replicate3,Div19_basal,GPR37L_1,8.0
3,Replicate4,Div19_basal,GPR37L_1,8.0
4,Replicate5,Div19_basal,GPR37L_1,8.0
...,...,...,...,...
955,Replicate20,Div19_20uMCNO,unt2,8.0
956,Replicate21,Div19_20uMCNO,unt2,8.0
957,Replicate22,Div19_20uMCNO,unt2,8.0
958,Replicate23,Div19_20uMCNO,unt2,8.0


In [27]:
testsI_long.keys()

dict_keys(['Number of Spikes', 'Mean Firing Rate (Hz)', 'ISI Coefficient of Variation - Avg', 'Number of Active Electrodes', 'Weighted Mean Firing Rate (Hz)', 'Number of Bursts', 'Number of Bursting Electrodes', 'Burst Duration - Avg (sec)', 'Burst Duration - Std (sec)', 'Number of Spikes per Burst - Avg', 'Number of Spikes per Burst - Std', 'Mean ISI within Burst - Avg (sec)', 'Mean ISI within Burst - Std (sec)', 'Median ISI within Burst - Avg (sec)', 'Median ISI within Burst - Std (sec)', 'Median/Mean ISI within Burst - Avg', 'Median/Mean ISI within Burst - Std', 'Inter-Burst Interval - Avg (sec)', 'Inter-Burst Interval - Std (sec)', 'Burst Frequency - Avg (Hz)', 'Burst Frequency - Std (Hz)', 'IBI Coefficient of Variation - Avg', 'IBI Coefficient of Variation - Std', 'Burst Percentage - Avg', 'Burst Percentage - Std', 'Number of Network Bursts', 'Network Burst Frequency', 'Network Burst Duration - Avg (sec)', 'Network Burst Duration - Std (sec)', 'Number of Spikes per Network Burst -

In [28]:
def categorize_gene_vs_control(testparam_df, control_var='unt2'):
    genes = list(testparam_df['Gene'].unique())
    mutants = [gene for gene in genes if "unt" not in gene]
    mutVcont_dict= {mut: pd.DataFrame for mut in mutants}
    for mutant in mutants:
        tiny_df = testparam_df[(testparam_df['Gene'] == control_var) |\
                   (testparam_df['Gene'] == mutant)]
        mutVcont_dict[mutant] = tiny_df
    return mutVcont_dict

## created from testsI_mapped
comparisons_long = {}
for test_param, table in testsI_long.items():
    comparisons_long[test_param] = categorize_gene_vs_control(table, control_var='unt2')
comparisons_long['Number of Active Electrodes']['GPR37L_1']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0
1,Replicate2,Div19_basal,GPR37L_1,8.0
2,Replicate3,Div19_basal,GPR37L_1,8.0
3,Replicate4,Div19_basal,GPR37L_1,8.0
4,Replicate5,Div19_basal,GPR37L_1,8.0
...,...,...,...,...
955,Replicate20,Div19_20uMCNO,unt2,8.0
956,Replicate21,Div19_20uMCNO,unt2,8.0
957,Replicate22,Div19_20uMCNO,unt2,8.0
958,Replicate23,Div19_20uMCNO,unt2,8.0


In [29]:
comparisons_long['Number of Active Electrodes']['GPR37L_1']['Subject'].unique()

array(['Replicate1', 'Replicate2', 'Replicate3', 'Replicate4',
       'Replicate5', 'Replicate6', 'Replicate7', 'Replicate8',
       'Replicate9', 'Replicate10', 'Replicate11', 'Replicate12',
       'Replicate13', 'Replicate14', 'Replicate15', 'Replicate16',
       'Replicate17', 'Replicate18', 'Replicate19', 'Replicate20',
       'Replicate21', 'Replicate22', 'Replicate23', 'Replicate24'],
      dtype=object)

In [30]:
### You want to troubleshoot w PRISM??

TESTPARAM = 'Area Under Normalized Cross-Correlation'
GENEINT = 'GPR37L_1'
# TestParamsSectioned_dict[TESTPARAM][GENEINT].to_csv(f"{parent}/data/PRISM_inputs/{TESTPARAM}_{GENEINT}.csv")

In [31]:
comparisons_long['Number of Active Electrodes']['GPR37L_1']

,Subject,Chemical_Type,Gene,Quantity
0,Replicate1,Div19_basal,GPR37L_1,8.0
1,Replicate2,Div19_basal,GPR37L_1,8.0
2,Replicate3,Div19_basal,GPR37L_1,8.0
3,Replicate4,Div19_basal,GPR37L_1,8.0
4,Replicate5,Div19_basal,GPR37L_1,8.0
...,...,...,...,...
955,Replicate20,Div19_20uMCNO,unt2,8.0
956,Replicate21,Div19_20uMCNO,unt2,8.0
957,Replicate22,Div19_20uMCNO,unt2,8.0
958,Replicate23,Div19_20uMCNO,unt2,8.0


In [32]:
def ensure_dirs_exists(path):
    directory = os.path.dirname(path)  # Get the directory part of the file path
    if not os.path.exists(directory):
        os.makedirs(directory)
    return

In [40]:
def calculate_ANOVA(testparameter, df, anovaRM=True,
                    control_var="unt2"):
    """_summary_
    In a two-way repeated anova measurement test, 1 factor== within-subject, each 
        subject is exposed to all levels of this factor (chemical_Type).
        -Value is dependent variable
        -Gene is index, each gene is eitehr control or mutation so it is key identifier
        -Chemical_type is change within each subject
    
    Args:
        testparameter (_type_): _description_from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

    Returns:
        Anova TABLE: showing mut (1) vs control (1), significance of gene
                        perturbation, gene&chemical
            'Source': Name of the within-group factor
        if RM:
            'ddof1': Degrees of freedom (numerator)

            'ddof2': Degrees of freedom (denominator)

            'F': F-value ## if null hypothesis is true, f to have val close to 1
                - variation between sample means/variation within the samples.

            'p-unc': Uncorrected p-value

            'ng2': Generalized eta-square effect size 
            - to understand magnitude of these differences. 
             - 0.02, 2.26% of variance in quanitty is caused by that.
             - small value indicates no individual factors have a large impact on
             dependent variable.

            'eps': Greenhouse-Geisser epsilon factor (= index of sphericity)

            'p-GG-corr': Greenhouse-Geisser corrected p-value

            'W-spher': Sphericity test statistic


            'p-spher': p-value of the sphericity test

            'sphericity': sphericity of the data (boolean)
                - variances between groups are equal. g1-g2 (+ all subjects) g1-g3. g2-g3. all equal?
        else:
            sum_sq: measure of total variation within data. how much data points differ from the mean
            df (deg of freedom): independ. values, values that are free to vary once we know
                            constraints (sample means)
            F-statistic:  model variance / residual variance. amount of variance due to the variable.
            P-value: is the variance noted (sum_sq, F-statistic) significant?
    """
    if anovaRM:
        # print(df.shape, df.head())
        anova_table = pg.rm_anova(dv='Quantity', within=['Chemical_Type', 'Gene'],
                  subject='Subject', data=df)
        anova_table.index = anova_table['Source']
    else:
        model = ols('Value ~ C(Gene) + C(Chemical_Type) + C(Gene):C(Chemical_Type)', data= df).fit()
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_table = anova_table.drop('Residual')
    return anova_table

ANOVA = {test_param: {} for test_param in comparisons_long.keys()}
for test_param, mutVcontrol_dict in comparisons_long.items():
    for mut, table in mutVcontrol_dict.items():
        if table['Chemical_Type'].nunique() > 1:
            ANOVA[test_param][mut]= calculate_ANOVA(test_param, table, anovaRM=ANOVARM).drop(columns='Source')

# ANOVA_dict['Mean Firing Rate (Hz)']['GPR37L_1']

/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/pingouin/distribution.py:742: RuntimeWarning: invalid value encountered in scalar divide
  eps = np.min([num / den, 1])
/home/poojaparameswaran/anaconda3/envs/anovaenv/lib/python3.8/site-packages/pingouin/distribution.py:742: RuntimeWarning: invalid value encountered in scalar divide
  eps = np.min([num / den, 1])


In [41]:
ANOVA['Mean Firing Rate (Hz)']['GPR37L_1']

,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
Source,,,,,,,,,
Chemical_Type,9.0e+01,4,92,2.3e+01,4.0e+00,4.7e-03,3.2e-02,6.5e-02,4.2e-01
Gene,5.0e+00,1,23,5.0e+00,5.2e-01,4.8e-01,4.8e-01,3.9e-03,1.0e+00
Chemical_Type * Gene,1.0e+01,4,92,2.5e+00,7.6e-01,5.5e-01,5.1e-01,7.7e-03,6.6e-01


In [42]:
ANOVA['Area Under Cross-Correlation']['SLITRK5']

,SS,ddof1,ddof2,MS,F,p-unc,p-GG-corr,ng2,eps
Source,,,,,,,,,
Chemical_Type,1.4e+03,4,92,3.5e+02,1.2e+00,3.2e-01,3.2e-01,2.2e-02,5.2e-01
Gene,1.1e-02,1,23,1.1e-02,3.4e-05,1.0e+00,1.0e+00,1.8e-07,1.0e+00
Chemical_Type * Gene,3.5e+02,4,92,8.7e+01,5.5e-01,7.0e-01,5.5e-01,5.6e-03,4.3e-01


In [43]:
def create_multind_df(ANOVA_dict):
    pd.set_option('display.float_format', lambda x: '%.1e' % x)
    test_params = list(ANOVA_dict.keys())
    mutant_genes = list(ANOVA_dict[test_params[0]].keys())
    anova_variables = list(ANOVA_dict[test_params[0]][mutant_genes[0]].index)
    col_iterables = [mutant_genes, ["F-stat", "p-GG-corr","Eta^2 effect size-ng2"]]
    row_iterables = [test_params, anova_variables]
    
    rindex = pd.MultiIndex.from_product(row_iterables, names=["Test Parameter", "ANOVA variables"])
    cindex = pd.MultiIndex.from_product(col_iterables)
    
    df = pd.DataFrame(index=rindex, columns= cindex)
    for test_param, testparam_dict in ANOVA_dict.items():
        for mutant, anova_df in testparam_dict.items():
            for var in anova_variables:
                df.loc[(test_param, var), (mutant, 'F-stat')] = anova_df.loc[var, 'F']
                df.loc[(test_param, var), (mutant, 'p-GG-corr')] = anova_df.loc[var, 'p-GG-corr']
                df.loc[(test_param, var), (mutant, 'Eta^2 effect size-ng2')] = anova_df.loc[var, 'ng2']
                # df.loc[(test_param, 'C(Gene)'), (mutant, 'F-statistic')] = (anova_df['F']['C(Gene)'])
    return df

nf= create_multind_df(ANOVA)

nf

GPR37L_1            \
                                                        F-stat p-GG-corr   
Test Parameter                   ANOVA variables                           
Number of Spikes                 Chemical_Type         4.0e+00   3.2e-02   
                                 Gene                  5.2e-01   4.8e-01   
                                 Chemical_Type * Gene  7.6e-01   5.1e-01   
Mean Firing Rate (Hz)            Chemical_Type         4.0e+00   3.2e-02   
                                 Gene                  5.2e-01   4.8e-01   
...                                                        ...       ...   
Number of Covered Electrodes     Gene                  7.8e-11   1.0e+00   
                                 Chemical_Type * Gene  7.8e-11       NaN   
Weighted Mean Resistance (kOhms) Chemical_Type         1.2e-27   1.0e+00   
                                 Gene                  5.3e+01   2.1e-07   
                                 Chemical_Type * Gene -4.4e-14   1.0e+00   

                                                                             \
                                                      Eta^2 effect size-ng2   
Test Parameter                   ANOVA variables                              
Number of Spikes                 Chemical_Type                      6.5e-02   
                                 Gene                               3.9e-03   
                                 Chemical_Type * Gene               7.7e-03   
Mean Firing Rate (Hz)            Chemical_Type                      6.5e-02   
                                 Gene                               3.9e-03   
...                                                                     ...   
Number of Covered Electrodes     Gene                               9.9e-29   
                                 Chemical_Type * Gene              -9.9e-29   
Weighted Mean Resistance (kOhms) Chemical_Type                      3.0e-29   
                                 Gene                               1.0e-01   
                                 Chemical_Type * Gene              -3.7e-16   

                                                      GPR37L_2            \
                                                        F-stat p-GG-corr   
Test Parameter                   ANOVA variables                           
Number of Spikes                 Chemical_Type         8.0e+00   8.9e-04   
                                 Gene                  3.8e+00   6.5e-02   
                                 Chemical_Type * Gene  1.1e+00   3.4e-01   
Mean Firing Rate (Hz)            Chemical_Type         8.0e+00   8.9e-04   
                                 Gene                  3.8e+00   6.5e-02   
...                                                        ...       ...   
Number of Covered Electrodes     Gene                  1.3e+01   1.5e-03   
                                 Chemical_Type * Gene  0.0e+00   1.0e+00   
Weighted Mean Resistance (kOhms) Chemical_Type         1.5e-27   1.0e+00   
                                 Gene                  7.8e+00   1.0e-02   
                                 Chemical_Type * Gene -1.4e-14   1.0e+00   

                                                                             \
                                                      Eta^2 effect size-ng2   
Test Parameter                   ANOVA variables                              
Number of Spikes                 Chemical_Type                      9.3e-02   
                                 Gene                               2.7e-02   
                                 Chemical_Type * Gene               1.4e-02   
Mean Firing Rate (Hz)            Chemical_Type                      9.3e-02   
                                 Gene                               2.7e-02   
...                                                                     ...   
Number of Covered Electrodes     Gene                               5.4e-02   
                           

In [44]:
## highlight rows when chemical and gene are significant.. 
def sigpval(p):
    lessthanp = p < 0.05
    return ['background-color: #CD5C5C' if v else '' for v in lessthanp]

psubset = [(col, 'p-GG-corr') for col in nf.columns.get_level_values(0).unique()]
msubset = [(col, 'Eta^2 effect size-ng2') for col in nf.columns.get_level_values(0).unique()]
fubset = [(col, 'F-stat') for col in nf.columns.get_level_values(0).unique()]
all = []
all.extend(psubset)
all.extend(msubset)
all.extend(fubset)

foldname = '-'.join(list(map(lambda x : x.split("_")[-1], folder_dict.keys())))

if ANOVARM:
    SAVE_ANOVA_TABLE_FILE = f"{parent}/ANOVA_results/2wayANOVARM/{PERTURBATION}/{foldname}/{EXPERIMENT_TIME}/{plate_type}_{PERTURBATION}_ANOVARMresults.xlsx"
else:
    SAVE_ANOVA_TABLE_FILE = f"{parent}/ANOVA_results/2wayANOVA/{PERTURBATION}/{foldname}/{EXPERIMENT_TIME}/{plate_type}_{PERTURBATION}_ANOVAresults.xlsx"
ensure_dirs_exists(SAVE_ANOVA_TABLE_FILE)
nf.style.apply(sigpval, subset=psubset).to_excel(SAVE_ANOVA_TABLE_FILE)


In [45]:
nf.style.apply(sigpval, subset=psubset)

In [46]:
nf['GPR37L_1']['F-stat']

Test Parameter                    ANOVA variables     
Number of Spikes                  Chemical_Type           4.0e+00
                                  Gene                    5.2e-01
                                  Chemical_Type * Gene    7.6e-01
Mean Firing Rate (Hz)             Chemical_Type           4.0e+00
                                  Gene                    5.2e-01
                                                           ...   
Number of Covered Electrodes      Gene                    7.8e-11
                                  Chemical_Type * Gene    7.8e-11
Weighted Mean Resistance (kOhms)  Chemical_Type           1.2e-27
                                  Gene                    5.3e+01
                                  Chemical_Type * Gene   -4.4e-14
Name: F-stat, Length: 165, dtype: object

In [47]:
def calculate_Tukeys(testparameter, df, show_barlabels = False, control_var="unt2"):  
    ## Perform Tukey's HSD test
    tukey = pairwise_tukeyhsd(endog=df['Value'],
                            groups=df['Gene'] + '_' + df['Chemical_Type'],
                            alpha=0.05)
    tukey_df = pd.DataFrame(data=tukey.summary().data[1:], columns=tukey.summary().data[0])
    tukey_control = tukey_df[tukey_df['group1'].str.contains('unt2') | tukey_df['group2'].str.contains('unt2')]
    mask = tukey_control['group1'].str.split('_').str[-1] == tukey_control['group2'].str.split('_').str[-1]
    matching_perturb_rows = tukey_control[mask]
    return matching_perturb_rows